In [52]:
import numpy as np
import pandas as pd
import os
import pickle
import xlsxwriter
from os.path import join as PT
import math

In [53]:
CODE = "32"
TREE_BASE_DIR = "Tree_Structure/"
SEGMENTS_BASE_DIR = "output_fixation/"+CODE+"/"+"freq_segment/data/"
OUTPUT_DIR = "output_fixation/"+CODE+"/"+"tree_segment/weighted/"
VISUAL_DIR = "weighted_visual_plots_new/"
THRESHOLD = 0.05
color_map = {0 : "#dfe7f9", 1 : "#dae5d5", 2 : "#fff2cc", 3 : "#f8cecc", 4: "#e1d5e7", 5: "#e1d537", 6 : "#60a917",
            7 : "#0050ef", 8 : "#a20025"}
color_map_code = {0 : "#d80073", 1 : "#aa00ff", 2 : "#647687", 3 : "#6d8764", 4: "#fa6800", 5: "#e1d537", 6 : "#60a917",
            7 : "#0050ef", 8 : "#a20025"}

In [54]:
if not os.path.exists(os.path.join(OUTPUT_DIR)):
    os.mkdir(os.path.join(OUTPUT_DIR))
if not os.path.exists(os.path.join(TREE_BASE_DIR,CODE)):
    os.mkdir(os.path.join(TREE_BASE_DIR,CODE))
if not os.path.exists(os.path.join(TREE_BASE_DIR,CODE,VISUAL_DIR)):
    os.mkdir(os.path.join(TREE_BASE_DIR,CODE,VISUAL_DIR))

In [55]:
class Node:
    def __init__(self,name=None,node_num=0):
        self.node_num = node_num
        self.name = name
        self.children = []
        self.lines = []
        
    def insert(self,child):
        self.children.append(child)
    
    def traverse(self):
        print(self)
        for child in self.children:
            child.traverse()
    
    def __str__(self):
        return "["+str(self.node_num)+" : "+self.name+" : "+str(self.lines)\
    +" : "+str([x.node_num for x in self.children])+"]"

In [56]:
def read_and_create_tree(file):
    with open(file) as f:
        nodes = []
        for line in f:
            if line[-1] == '\n':
                line = line[:-1]
            statements = line.split(" ")
            if statements[0] == "CREATE":
                node = Node(statements[1],len(nodes))
                num_lines = int(statements[2])
                for i in range(num_lines):
                    node.lines.append(int(statements[i+3]))
                nodes.append(node)
            elif statements[0] == "INSERT":
                parent = int(statements[1])
                child = int(statements[2])
                nodes[parent].insert(nodes[child])
            else:
                print("Error: Invalid command!!")
                break
        return nodes[0]

In [57]:
def get_line_to_node_mapping(root,mappings):
    for line in root.lines:
        if line in mappings:
            print("Error : Line "+str(line)+" mapped twice")
            return
        mappings[line] = root
    for child in root.children:
        get_line_to_node_mapping(child,mappings)
    return

def init_node_counts(root,counts):
    counts[root] = 0
    for child in root.children:
        init_node_counts(child,counts)
        
def get_nodes(root,all_nodes):
    all_nodes[root.node_num] = root
    for child in root.children:
        get_nodes(child,all_nodes)
        
def get_major_components(data):
    major = []
    data = np.array(data)
    total = np.sum(data[:,1])
    for row in data:
        if row[1]/total >= THRESHOLD:
            major.append(row[0])
    return major

def update_node_counts(root,node_counts,major_lines,node_weights):
    for child in root.children:
        update_node_counts(child,node_counts,major_lines,node_weights)
    to_add = False
    for line in root.lines:
        if line in major_lines:
            to_add = True
            break
    if to_add:
        node_counts[root] = node_counts[root] + 1
    if len(root.lines) == 0:
        tot = 0
        for child in root.children:
            tot += node_counts[child] * node_weights[child][0]
        node_counts[root] = tot
        
def find_parents(root,prevNode,parents):
    parents[root] = prevNode
    for child in root.children:
        find_parents(child,root,parents)

def form_node_weights(root,line_weights,node_weights):
    if len(root.children) == 0:
        tot = 0
        for line in root.lines:
            tot += line_weights[line-1]
        node_weights[root] = (tot/len(root.lines),len(root.lines))
    else:
        tot = 0
        tot_lines = 0
        for child in root.children:
            form_node_weights(child,line_weights,node_weights)
            tot += node_weights[child][0]*node_weights[child][1]
            tot_lines += node_weights[child][1]
        node_weights[root] = (tot/tot_lines,tot_lines)        

In [58]:
root = read_and_create_tree(os.path.join(TREE_BASE_DIR,CODE,CODE+".tree"))
all_nodes = {}
get_nodes(root,all_nodes)
all_nodes

{0: <__main__.Node at 0x7f8aacd476a0>,
 1: <__main__.Node at 0x7f8aacd47160>,
 2: <__main__.Node at 0x7f8aacd47128>,
 3: <__main__.Node at 0x7f8aacd94860>,
 4: <__main__.Node at 0x7f8aacd947f0>,
 5: <__main__.Node at 0x7f8aacd94780>,
 6: <__main__.Node at 0x7f8aacd4cf28>,
 7: <__main__.Node at 0x7f8aacd4c588>,
 8: <__main__.Node at 0x7f8aacd4cc88>,
 9: <__main__.Node at 0x7f8aacd4c3c8>,
 10: <__main__.Node at 0x7f8aacd4c4a8>}

In [59]:
root.traverse()

[0 : root : [] : [1, 2]]
[1 : func-displayList : [] : [3, 4]]
[3 : statement : [3, 4, 5, 6] : []]
[4 : loop : [7, 8] : []]
[2 : func-main : [] : [5, 6]]
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []]
[6 : loop : [] : [7, 8]]
[7 : statement : [21, 22] : []]
[8 : loop : [] : [9, 10]]
[9 : statement : [23, 24] : []]
[10 : if : [25, 26, 27, 28, 29, 30] : []]


In [60]:
'''
line_weights = []
with open(PT(TREE_BASE_DIR,CODE,CODE+"_complexity.txt")) as f:
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line_weights.append(float(line))
line_weights
'''

'\nline_weights = []\nwith open(PT(TREE_BASE_DIR,CODE,CODE+"_complexity.txt")) as f:\n    for line in f:\n        if line[-1] == \'\n\':\n            line = line[:-1]\n        line_weights.append(float(line))\nline_weights\n'

In [61]:
stTypetoValue = {}
st_wt = pd.read_excel(PT(TREE_BASE_DIR,CODE,"weighted_data","statement_weight.xlsx"))
st_wt = np.array(st_wt)
for x in st_wt:
    stTypetoValue[x[0]] = x[1]

line_weights = np.zeros(100)
valtoLine = pd.read_csv(PT(TREE_BASE_DIR,CODE,"weighted_data","code"+CODE+"_fraw.csv"))
valtoLine = np.array(valtoLine)
for x in valtoLine:
    if x[0] in stTypetoValue:
        for i in range(x[1],x[2]+1):
            line_weights[i-1] = max(line_weights[i-1],stTypetoValue[x[0]])
line_weights

array([0. , 0. , 2. , 0. , 1. , 3. , 4. , 3. , 3. , 0. , 2. , 0. , 3. ,
       1. , 1. , 1. , 0. , 4. , 0. , 4. , 0. , 2. , 0. , 3. , 3. , 3. ,
       0.5, 0. , 0. , 3. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

In [62]:
node_weights = {}
form_node_weights(root,line_weights,node_weights)
for k,v in node_weights.items():
    print(k,v)
ROOT_WEIGHT = node_weights[all_nodes[0]][0]
print(ROOT_WEIGHT)

[3 : statement : [3, 4, 5, 6] : []] (1.5, 4)
[4 : loop : [7, 8] : []] (3.5, 2)
[1 : func-displayList : [] : [3, 4]] (2.1666666666666665, 6)
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] (1.6, 10)
[7 : statement : [21, 22] : []] (1.0, 2)
[9 : statement : [23, 24] : []] (1.5, 2)
[10 : if : [25, 26, 27, 28, 29, 30] : []] (1.5833333333333333, 6)
[8 : loop : [] : [9, 10]] (1.5625, 8)
[6 : loop : [] : [7, 8]] (1.45, 10)
[2 : func-main : [] : [5, 6]] (1.525, 20)
[0 : root : [] : [1, 2]] (1.6730769230769231, 26)
1.6730769230769231


In [63]:
for file in os.listdir(os.path.join(SEGMENTS_BASE_DIR)):
    print(file)
    xls = pd.read_excel(os.path.join(SEGMENTS_BASE_DIR,file),sheetname=None)
    segment_status = []
    node_counts = {}
    init_node_counts(root,node_counts)
    for segment,data in xls.items():
        major_lines = get_major_components(data)
        print(major_lines)
        update_node_counts(root,node_counts,major_lines,node_weights)
        segment_status.append(dict(node_counts))
    with open(os.path.join(OUTPUT_DIR,file[:-4]+"_segment_status.pkl"),'wb') as f:
        pickle.dump(segment_status,f)

final_matrix_newfix_vishal3.xlsx
[1, 2, 3, 5, 7, 13, 16]
[5, 6, 7, 9, 10, 12, 13, 15]
[18, 22, 24, 25]
[11, 14, 19]
final_matrix_newfix_shweta3.xlsx
[1, 2, 6, 7, 8, 10]
[13, 14, 16, 17, 18]
[22, 23, 24, 25]
[2, 3, 6, 7, 8]
[16, 17, 19, 20, 21, 22, 24, 29]
final_matrix_newfix_achal3.xlsx
[1, 2]
[14, 15, 16, 17, 19]
[3, 5, 6, 7, 8]
[13, 14, 15, 16, 23, 24, 25]
[13, 16, 17, 18, 19, 20, 21, 22, 25, 26, 28, 29]
[7, 10, 18, 20, 21, 22, 23, 24]
[18, 19, 20, 21, 23, 29, 30]
[17, 18, 19, 20, 21, 22]
[2, 8, 15, 17]
final_matrix_newfix_vrajesh3.xlsx
[4, 9, 10, 11, 19, 21]
[6, 7, 14, 15, 16, 17, 24, 25, 26, 28]
[7, 17, 18, 19, 21]
[9, 13, 14, 15]
[9, 30]
final_matrix_newfix_arihant3.xlsx
[2, 3, 5, 6, 7, 8]
[9, 12, 13, 14, 15, 16, 17]
[6, 7, 8]
[8, 17, 18, 22, 25]
[19, 20, 21, 22]
[17, 18, 19, 20, 21, 23]
[21, 22, 23, 24, 25, 30]
[6, 7, 13, 14, 15, 16, 19, 22]
[2, 3, 5, 8]
final_matrix_newfix_aditya3.xlsx
[1, 2, 3, 5, 6, 7, 8]
[13, 14, 16, 17, 22, 23, 24]
[6, 7, 8]
[13, 14, 15, 16]
[17, 18, 19, 24,

In [64]:
for file in os.listdir(os.path.join(OUTPUT_DIR)):
    print("Subject - ",file)
    with open(os.path.join(OUTPUT_DIR,file),'rb') as f:
        seg_stats = pickle.load(f)
    for i,seg_stat in enumerate(seg_stats):
        print("Segment : ",str(i+1))
        for node,ct in seg_stat.items():
            print(node,ct)
    print("==============================================================================")

Subject -  final_matrix_newfix_arihant3._segment_status.pkl
Segment :  1
[0 : root : [] : [1, 2]] 10.833333333333332
[1 : func-displayList : [] : [3, 4]] 5.0
[3 : statement : [3, 4, 5, 6] : []] 1
[4 : loop : [7, 8] : []] 1
[2 : func-main : [] : [5, 6]] 0.0
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 0
[6 : loop : [] : [7, 8]] 0.0
[7 : statement : [21, 22] : []] 0
[8 : loop : [] : [9, 10]] 0.0
[9 : statement : [23, 24] : []] 0
[10 : if : [25, 26, 27, 28, 29, 30] : []] 0
Segment :  2
[0 : root : [] : [1, 2]] 13.273333333333332
[1 : func-displayList : [] : [3, 4]] 5.0
[3 : statement : [3, 4, 5, 6] : []] 1
[4 : loop : [7, 8] : []] 1
[2 : func-main : [] : [5, 6]] 1.6
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 1
[6 : loop : [] : [7, 8]] 0.0
[7 : statement : [21, 22] : []] 0
[8 : loop : [] : [9, 10]] 0.0
[9 : statement : [23, 24] : []] 0
[10 : if : [25, 26, 27, 28, 29, 30] : []] 0
Segment :  3
[0 : root : [] : [1, 2]] 24.106666666666666
[1 : func-displ

# Tree Graph Generation

In [65]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [66]:
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout
G = nx.Graph()

In [67]:
def read_and_create_graph(file,node_weights,all_nodes):
    G = nx.DiGraph()
    terminal_nodes = []
    labels = {}
    with open(file) as f:
        ct = 0
        for line in f:
            if line[-1] == '\n':
                line = line[:-1]
            statements = line.split(" ")
            if statements[0] == "CREATE":
                label = statements[1]
                G.add_node(ct)                
                num_lines = int(statements[2])
                
                if num_lines != 0:
                    terminal_nodes.append(ct)
                    label += '\n'
                    label += statements[3]
                    label += " - "
                    label += statements[-1]
                label += '\nWeight: '
                label += '{0:.2f}'.format(node_weights[all_nodes[ct]][0])
                labels[ct] = label
                ct += 1
            elif statements[0] == "INSERT":
                parent = int(statements[1])
                child = int(statements[2])
                G.add_edge(parent,child)
                
                #nodes[parent].insert(nodes[child])
            else:
                print("Error: Invalid command!!")
                break
        return G, terminal_nodes, ct, labels

In [68]:
G, terminal_nodes, total_nodes, labels = read_and_create_graph(os.path.join(TREE_BASE_DIR,CODE,CODE+".tree"),
                                                               node_weights,all_nodes)
node_color = [('#dae5d5' if node in terminal_nodes else '#dfe7f9') for node in range(total_nodes)]

In [69]:
plt.figure(figsize=(20,15))
pos=graphviz_layout(G, prog='dot')
nx.draw(G,pos,arrows=False,node_size=20000,node_color=node_color,labels=labels,font_size=20)
plt.suptitle("Code %s Tree" % str(CODE),fontsize = 36)
plt.savefig(os.path.join(TREE_BASE_DIR,CODE,VISUAL_DIR,CODE+".png"))

/home/ayush/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [70]:
patches = []
num_colors = len(color_map)
for i in range(num_colors-1):
    pt = mpatches.Patch(color=color_map[i],label=str(i))
    patches.append(pt)
patches.append(mpatches.Patch(color=color_map[num_colors-1],label=str(num_colors-1)+"+"))

In [71]:
for file in os.listdir(os.path.join(OUTPUT_DIR)):
    sub_name = file[:-20]
    if not os.path.exists(os.path.join(TREE_BASE_DIR,CODE,VISUAL_DIR,sub_name)):
        os.mkdir(os.path.join(TREE_BASE_DIR,CODE,VISUAL_DIR,sub_name))
    print("Subject - ",sub_name)
    with open(os.path.join(OUTPUT_DIR,file),'rb') as f:
        seg_stats = pickle.load(f)
    for i,seg_stat in enumerate(seg_stats):
        seg_labels = dict(labels)
        for node, label in seg_labels.items():
            seg_labels[node] += "\n"
        segment_num = str(i+1)
        print("Segment : ",segment_num)
        for node,ct in seg_stat.items():
            node_num = node.node_num
            seg_labels[node_num] += "Count : " + '{0:.2f}'.format(ct)
            if int(ct) in color_map:
                node_color[node_num] = color_map[int(ct)]
            else:
                node_color[node_num] = color_map[len(color_map)-1]
            print(node,ct)
        plt.figure(figsize=(20,15))
        pos=graphviz_layout(G, prog='dot')
        nx.draw(G,pos,arrows=False,node_size=20000,node_color=node_color,labels=seg_labels,font_size=22)
        plt.suptitle("Segment %d" % (i+1),fontsize = 36)
        plt.legend(handles=patches,fontsize=24)
        plt.savefig(os.path.join(TREE_BASE_DIR,CODE,VISUAL_DIR,sub_name,segment_num+".png"))
    print("==============================================================================")

Subject -  final_matrix_newfix_arihant3
Segment :  1
[0 : root : [] : [1, 2]] 10.833333333333332
[1 : func-displayList : [] : [3, 4]] 5.0
[3 : statement : [3, 4, 5, 6] : []] 1
[4 : loop : [7, 8] : []] 1
[2 : func-main : [] : [5, 6]] 0.0
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 0
[6 : loop : [] : [7, 8]] 0.0
[7 : statement : [21, 22] : []] 0
[8 : loop : [] : [9, 10]] 0.0
[9 : statement : [23, 24] : []] 0
[10 : if : [25, 26, 27, 28, 29, 30] : []] 0


/home/ayush/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Segment :  2
[0 : root : [] : [1, 2]] 13.273333333333332
[1 : func-displayList : [] : [3, 4]] 5.0
[3 : statement : [3, 4, 5, 6] : []] 1
[4 : loop : [7, 8] : []] 1
[2 : func-main : [] : [5, 6]] 1.6
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 1
[6 : loop : [] : [7, 8]] 0.0
[7 : statement : [21, 22] : []] 0
[8 : loop : [] : [9, 10]] 0.0
[9 : statement : [23, 24] : []] 0
[10 : if : [25, 26, 27, 28, 29, 30] : []] 0
Segment :  3
[0 : root : [] : [1, 2]] 24.106666666666666
[1 : func-displayList : [] : [3, 4]] 10.0
[3 : statement : [3, 4, 5, 6] : []] 2
[4 : loop : [7, 8] : []] 2
[2 : func-main : [] : [5, 6]] 1.6
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 1
[6 : loop : [] : [7, 8]] 0.0
[7 : statement : [21, 22] : []] 0
[8 : loop : [] : [9, 10]] 0.0
[9 : statement : [23, 24] : []] 0
[10 : if : [25, 26, 27, 28, 29, 30] : []] 0
Segment :  4
[0 : root : [] : [1, 2]] 41.81179036458333
[1 : func-displayList : [] : [3, 4]] 13.5
[3 : statement : [3, 4, 5, 6] : [

Subject -  final_matrix_newfix_achal3
Segment :  1
[0 : root : [] : [1, 2]] 0.0
[1 : func-displayList : [] : [3, 4]] 0.0
[3 : statement : [3, 4, 5, 6] : []] 0
[4 : loop : [7, 8] : []] 0
[2 : func-main : [] : [5, 6]] 0.0
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 0
[6 : loop : [] : [7, 8]] 0.0
[7 : statement : [21, 22] : []] 0
[8 : loop : [] : [9, 10]] 0.0
[9 : statement : [23, 24] : []] 0
[10 : if : [25, 26, 27, 28, 29, 30] : []] 0
Segment :  2
[0 : root : [] : [1, 2]] 2.44
[1 : func-displayList : [] : [3, 4]] 0.0
[3 : statement : [3, 4, 5, 6] : []] 0
[4 : loop : [7, 8] : []] 0
[2 : func-main : [] : [5, 6]] 1.6
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 1
[6 : loop : [] : [7, 8]] 0.0
[7 : statement : [21, 22] : []] 0
[8 : loop : [] : [9, 10]] 0.0
[9 : statement : [23, 24] : []] 0
[10 : if : [25, 26, 27, 28, 29, 30] : []] 0
Segment :  3
[0 : root : [] : [1, 2]] 13.273333333333332
[1 : func-displayList : [] : [3, 4]] 5.0
[3 : statement : [3, 4, 5

Segment :  3
[0 : root : [] : [1, 2]] 41.85107421875
[1 : func-displayList : [] : [3, 4]] 10.0
[3 : statement : [3, 4, 5, 6] : []] 2
[4 : loop : [7, 8] : []] 2
[2 : func-main : [] : [5, 6]] 13.235677083333334
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 3
[6 : loop : [] : [7, 8]] 5.817708333333333
[7 : statement : [21, 22] : []] 1
[8 : loop : [] : [9, 10]] 3.083333333333333
[9 : statement : [23, 24] : []] 1
[10 : if : [25, 26, 27, 28, 29, 30] : []] 1
Segment :  4
[0 : root : [] : [1, 2]] 44.29107421875
[1 : func-displayList : [] : [3, 4]] 10.0
[3 : statement : [3, 4, 5, 6] : []] 2
[4 : loop : [7, 8] : []] 2
[2 : func-main : [] : [5, 6]] 14.835677083333334
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 4
[6 : loop : [] : [7, 8]] 5.817708333333333
[7 : statement : [21, 22] : []] 1
[8 : loop : [] : [9, 10]] 3.083333333333333
[9 : statement : [23, 24] : []] 1
[10 : if : [25, 26, 27, 28, 29, 30] : []] 1
Subject -  final_matrix_newfix_anjali3
Segment :  1


# Excel Genrator

In [74]:
workbook = xlsxwriter.Workbook(os.path.join(TREE_BASE_DIR,CODE,CODE+"_node_data_wegihted.xlsx"))
worksheet = workbook.add_worksheet()
cell_format = workbook.add_format({'align': 'center','valign' : 'centre'})
worksheet.write('A1',"S.No.",cell_format)
worksheet.write('B1',"Subjects",cell_format)
worksheet.write('C1',"No. of \nSegments",cell_format)
worksheet.set_row(0,30)
worksheet.set_column(2,2,30)

j = 0
nodenum_to_col_mapping = {}
for node_num, node in all_nodes.items():
    if len(node.children) == 0:
        continue
    worksheet.write(0,3+j,node.name,cell_format)
    nodenum_to_col_mapping[node_num] = 3+j
    j += 1

for i, file in enumerate(os.listdir(os.path.join(OUTPUT_DIR))):
    sub_name = file[20:-20]
    print("Subject - ",sub_name)
    print("Last Segment Details")
    with open(os.path.join(OUTPUT_DIR,file),'rb') as f:
        seg_stats = pickle.load(f)
    seg_stat = seg_stats[-1]
    num_segs = len(seg_stats)
    row_data = [str(i+1)+".",sub_name,str(num_segs)]
    worksheet.write_row(1+i,0,row_data)
    for node,ct in seg_stat.items():
        print(node, ct)
        sub_norm_ct = ct/num_segs
        root_norm_ct = sub_norm_ct/ROOT_WEIGHT
        if len(node.children) == 0:
            continue
        worksheet.write(1+i,nodenum_to_col_mapping[node.node_num],'{0:.2f}'.format(root_norm_ct))

num_subs = len(os.listdir(os.path.join(OUTPUT_DIR)))
worksheet.write(num_subs+3,0,"Complexity of Root")
worksheet.write(num_subs+3,2,str(ROOT_WEIGHT))
workbook.close()
        

    
    
    

Subject -  arihant3
Last Segment Details
[0 : root : [] : [1, 2]] 95.47923177083332
[1 : func-displayList : [] : [3, 4]] 23.5
[3 : statement : [3, 4, 5, 6] : []] 4
[4 : loop : [7, 8] : []] 5
[2 : func-main : [] : [5, 6]] 29.221354166666664
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 5
[6 : loop : [] : [7, 8]] 14.635416666666666
[7 : statement : [21, 22] : []] 5
[8 : loop : [] : [9, 10]] 6.166666666666666
[9 : statement : [23, 24] : []] 2
[10 : if : [25, 26, 27, 28, 29, 30] : []] 2
Subject -  vrajesh3
Last Segment Details
[0 : root : [] : [1, 2]] 51.972864583333326
[1 : func-displayList : [] : [3, 4]] 10.0
[3 : statement : [3, 4, 5, 6] : []] 2
[4 : loop : [7, 8] : []] 2
[2 : func-main : [] : [5, 6]] 19.872916666666665
[5 : statement : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] : []] 4
[6 : loop : [] : [7, 8]] 9.291666666666666
[7 : statement : [21, 22] : []] 2
[8 : loop : [] : [9, 10]] 4.666666666666666
[9 : statement : [23, 24] : []] 1
[10 : if : [25, 26, 27, 28, 29, 